In [1]:
# Imports

import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

from wrangle import acquire_zillow, get_zillow_data, wrangle_zillow, split_train_val_test

from explore import plot_variable_pairs, plot_categorical_and_continuous_vars

from features import make_features, size_cat, age_cat, simple_features

import os

In [2]:
df = get_zillow_data()
df = wrangle_zillow(df)
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,fips
4,4,2.0,3633,296425,2005,6037
6,3,4.0,1620,847770,2011,6037
7,3,2.0,2077,646760,1926,6037
11,0,0.0,1200,5328,1972,6037
14,0,0.0,171,6920,1973,6037


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2134909 entries, 4 to 2152862
Data columns (total 14 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   bedrooms                   int64   
 1   bathrooms                  float64 
 2   area                       int64   
 3   tax_value                  int64   
 4   year_built                 int64   
 5   total_rooms                float64 
 6   property_age               int64   
 7   county_6037                bool    
 8   county_6059                bool    
 9   county_6111                bool    
 10  size_per_bedroom           float64 
 11  bathroom_to_bedroom_ratio  float64 
 12  property_size_category     category
 13  property_age_group         category
dtypes: bool(3), category(2), float64(4), int64(5)
memory usage: 173.1 MB


In [10]:
df.isna().sum()

bedrooms                     0
bathrooms                    0
area                         0
tax_value                    0
year_built                   0
total_rooms                  0
property_age                 0
county_6037                  0
county_6059                  0
county_6111                  0
size_per_bedroom             0
bathroom_to_bedroom_ratio    0
property_size_category       0
property_age_group           0
dtype: int64

In [11]:
df.eq(0).sum()

bedrooms                           0
bathrooms                          0
area                               0
tax_value                          0
year_built                         0
total_rooms                        0
property_age                       0
county_6037                   713817
county_6059                  1583195
county_6111                  1972806
size_per_bedroom                   0
bathroom_to_bedroom_ratio          0
property_size_category             0
property_age_group                 0
dtype: int64

In [6]:
# Assuming your original DataFrame is named 'df'
# Create a new DataFrame containing rows with 'bedrooms' or 'bathrooms' equal to 0
df_zeros = df[(df['bedrooms'] == 0) | (df['bathrooms'] == 0)]
df_zeros.head()


,bedrooms,bathrooms,area,tax_value,year_built,taxamount,fips
11,0,0.0,1200,5328,1972,91.60,6037
14,0,0.0,171,6920,1973,255.17,6037
15,0,0.0,203,14166,1960,163.79,6037
398,3,0.0,2239,396608,1988,4404.22,6111
753,3,0.0,2421,1111120,1976,12394.16,6111


In [7]:
df_zeros.describe()

,bedrooms,bathrooms,area,tax_value,year_built,taxamount,fips
count,5326.000000,5326.000000,5326.000000,5.326000e+03,5326.000000,5326.000000,5326.000000
mean,0.531731,0.301915,1469.616598,3.662318e+05,1953.956628,4448.144921,6050.476906
std,1.296158,0.970216,7620.520046,1.338747e+06,27.104891,15710.931299,27.242037
min,0.000000,0.000000,1.000000,2.200000e+01,1890.000000,6.340000,6037.000000
25%,0.000000,0.000000,380.000000,3.011875e+04,1930.000000,504.845000,6037.000000
50%,0.000000,0.000000,728.000000,1.239035e+05,1953.000000,1723.285000,6037.000000
75%,0.000000,0.000000,1570.000000,3.217245e+05,1978.000000,4048.517500,6037.000000
max,18.000000,20.000000,531118.000000,3.336760e+07,2016.000000,399369.470000,6111.000000


In [8]:
df = df[(df['bedrooms'] != 0) & (df['bathrooms'] != 0)]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2134909 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   bedrooms    int64  
 1   bathrooms   float64
 2   area        int64  
 3   tax_value   int64  
 4   year_built  int64  
 5   taxamount   float64
 6   fips        int64  
dtypes: float64(2), int64(5)
memory usage: 130.3 MB


In [10]:
# Assuming df is your original DataFrame
df['total_rooms'] = df['bedrooms'] + df['bathrooms']

current_year = datetime.datetime.now().year
df['property_age'] = current_year - df['year_built']

#df['price_per_sqft'] = df['tax_value'] / df['area']

#df['tax_rate'] = df['taxamount'] / df['tax_value']

df = pd.get_dummies(df, columns=['fips'], prefix='county')

df['size_per_bedroom'] = df['area'] / df['bedrooms']

df['bathroom_to_bedroom_ratio'] = df['bathrooms'] / df['bedrooms']

df.head()

,bedrooms,bathrooms,area,tax_value,year_built,taxamount,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio
4,4,2.0,3633,296425,2005,6941.39,6.0,18,True,False,False,908.250000,0.500000
6,3,4.0,1620,847770,2011,10244.94,7.0,12,True,False,False,540.000000,1.333333
7,3,2.0,2077,646760,1926,7924.68,5.0,97,True,False,False,692.333333,0.666667
18,3,1.0,1244,169471,1950,2532.88,4.0,73,True,False,False,414.666667,0.333333
19,3,2.0,1300,233266,1950,3110.99,5.0,73,True,False,False,433.333333,0.666667


In [11]:
property_age_mean = df['property_age'].mean()
property_age_std = df['property_age'].std()

# Add corrected age range information for each category
age_ranges = {
    'very_new': f'<= {int(property_age_mean - 2 * property_age_std)} years',
    'new': f'{int(property_age_mean - 2 * property_age_std) + 1} - {int(property_age_mean - property_age_std)} years',
    'mid-aged': f'{int(property_age_mean - property_age_std) + 1} - {int(property_age_mean + property_age_std)} years',
    'old': f'{int(property_age_mean + property_age_std) + 1} - {int(property_age_mean + 2 * property_age_std)} years',
    'very_old': f'>{int(property_age_mean + 2 * property_age_std)} years'
}
age_ranges

{'very_new': '<= 17 years',
 'new': '18 - 39 years',
 'mid-aged': '40 - 84 years',
 'old': '85 - 106 years',
 'very_old': '>106 years'}

In [12]:
bins = [-float('inf'), property_age_mean - 2 * property_age_std, property_age_mean - property_age_std, property_age_mean + property_age_std, property_age_mean + 2 * property_age_std, float('inf')]

labels = ['very_new', 'new', 'mid-aged', 'old', 'very_old']

df['property_age_group'] = pd.cut(df['property_age'], bins=bins, labels=labels)

df.head()

,bedrooms,bathrooms,area,tax_value,year_built,taxamount,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio,property_age_group
4,4,2.0,3633,296425,2005,6941.39,6.0,18,True,False,False,908.250000,0.500000,new
6,3,4.0,1620,847770,2011,10244.94,7.0,12,True,False,False,540.000000,1.333333,very_new
7,3,2.0,2077,646760,1926,7924.68,5.0,97,True,False,False,692.333333,0.666667,old
18,3,1.0,1244,169471,1950,2532.88,4.0,73,True,False,False,414.666667,0.333333,mid-aged
19,3,2.0,1300,233266,1950,3110.99,5.0,73,True,False,False,433.333333,0.666667,mid-aged


In [13]:
area_mean = df['area'].mean()
area_std = df['area'].std()

small_range = (0, area_mean - area_std)
medium_range = (area_mean - area_std, area_mean + area_std)
large_range = (area_mean + area_std, np.inf)  # No upper limit for 'large'

print('Small:', small_range)
print('Medium:', medium_range)
print('Large:', large_range)

Small: (0, 701.7763344810669)
Medium: (701.7763344810669, 3026.575336714286)
Large: (3026.575336714286, inf)


In [14]:
# Create the 'property_size_category' column based on the area ranges
df['property_size_category'] = pd.cut(df['area'], bins=[-np.inf, small_range[1], large_range[0], np.inf], labels=['small', 'medium', 'large'])


In [15]:
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,taxamount,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio,property_age_group,property_size_category
4,4,2.0,3633,296425,2005,6941.39,6.0,18,True,False,False,908.250000,0.500000,new,large
6,3,4.0,1620,847770,2011,10244.94,7.0,12,True,False,False,540.000000,1.333333,very_new,medium
7,3,2.0,2077,646760,1926,7924.68,5.0,97,True,False,False,692.333333,0.666667,old,medium
18,3,1.0,1244,169471,1950,2532.88,4.0,73,True,False,False,414.666667,0.333333,mid-aged,medium
19,3,2.0,1300,233266,1950,3110.99,5.0,73,True,False,False,433.333333,0.666667,mid-aged,medium


In [16]:
# Calculate the percentile values for 25% and 75% of the 'tax_value' column
percentile_25 = np.percentile(df['tax_value'], 25)
percentile_75 = np.percentile(df['tax_value'], 75)

# Define the range intervals for 'low,' 'medium,' and 'high' price tiers
low_range = (-np.inf, percentile_25)
medium_range = (percentile_25, percentile_75)
high_range = (percentile_75, np.inf)

print('Low:', low_range)
print('Medium:', medium_range)
print('High:', high_range)


Low: (-inf, 189628.0)
Medium: (189628.0, 535000.0)
High: (535000.0, inf)


In [17]:
# Create the 'price_tier' column based on percentiles
#df['price_tier'] = pd.cut(df['tax_value'], bins=[-np.inf, percentile_25, percentile_75, np.inf], labels=['low', 'medium', 'high'])


In [ ]:
df = simple_features(df)

In [ ]:
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio
4,4,2.0,3633,296425,2005,6.0,18,True,False,False,908.250000,0.500000
6,3,4.0,1620,847770,2011,7.0,12,True,False,False,540.000000,1.333333
7,3,2.0,2077,646760,1926,5.0,97,True,False,False,692.333333,0.666667
18,3,1.0,1244,169471,1950,4.0,73,True,False,False,414.666667,0.333333
19,3,2.0,1300,233266,1950,5.0,73,True,False,False,433.333333,0.666667


In [ ]:
df = size_cat(df)
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio,property_size_category
4,4,2.0,3633,296425,2005,6.0,18,True,False,False,908.250000,0.500000,large
6,3,4.0,1620,847770,2011,7.0,12,True,False,False,540.000000,1.333333,medium
7,3,2.0,2077,646760,1926,5.0,97,True,False,False,692.333333,0.666667,medium
18,3,1.0,1244,169471,1950,4.0,73,True,False,False,414.666667,0.333333,medium
19,3,2.0,1300,233266,1950,5.0,73,True,False,False,433.333333,0.666667,medium


In [ ]:
df = age_cat(df)
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio,property_size_category,property_age_group
4,4,2.0,3633,296425,2005,6.0,18,True,False,False,908.250000,0.500000,large,new
6,3,4.0,1620,847770,2011,7.0,12,True,False,False,540.000000,1.333333,medium,very_new
7,3,2.0,2077,646760,1926,5.0,97,True,False,False,692.333333,0.666667,medium,old
18,3,1.0,1244,169471,1950,4.0,73,True,False,False,414.666667,0.333333,medium,mid-aged
19,3,2.0,1300,233266,1950,5.0,73,True,False,False,433.333333,0.666667,medium,mid-aged


In [3]:
df = make_features(df)

In [4]:
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio,property_age_group,property_size_category
4,4,2.0,3633,296425,2005,6.0,18,True,False,False,908.250000,0.500000,new,large
6,3,4.0,1620,847770,2011,7.0,12,True,False,False,540.000000,1.333333,very_new,medium
7,3,2.0,2077,646760,1926,5.0,97,True,False,False,692.333333,0.666667,old,medium
18,3,1.0,1244,169471,1950,4.0,73,True,False,False,414.666667,0.333333,mid-aged,medium
19,3,2.0,1300,233266,1950,5.0,73,True,False,False,433.333333,0.666667,mid-aged,medium


In [20]:
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio,property_size_category_small,property_size_category_medium,property_size_category_large,property_age_group_very_new,property_age_group_new,property_age_group_mid-aged,property_age_group_old,property_age_group_very_old
4,4,2.0,3633,296425,2005,6.0,18,True,False,False,908.250000,0.500000,False,False,True,False,True,False,False,False
6,3,4.0,1620,847770,2011,7.0,12,True,False,False,540.000000,1.333333,False,True,False,True,False,False,False,False
7,3,2.0,2077,646760,1926,5.0,97,True,False,False,692.333333,0.666667,False,True,False,False,False,False,True,False
18,3,1.0,1244,169471,1950,4.0,73,True,False,False,414.666667,0.333333,False,True,False,False,False,True,False,False
19,3,2.0,1300,233266,1950,5.0,73,True,False,False,433.333333,0.666667,False,True,False,False,False,True,False,False


In [5]:
df = pd.get_dummies(df, columns=['property_size_category', 'property_age_group'])

In [6]:
mms = MinMaxScaler()

to_scale = df.drop(columns=['tax_value' ]).columns

df[to_scale] = mms.fit_transform(df[to_scale])

df.head()

,bedrooms,bathrooms,area,tax_value,year_built,total_rooms,property_age,county_6037,county_6059,county_6111,size_per_bedroom,bathroom_to_bedroom_ratio,property_size_category_small,property_size_category_medium,property_size_category_large,property_age_group_very_new,property_age_group_new,property_age_group_mid-aged,property_age_group_old,property_age_group_very_old
4,0.125000,0.047619,0.003813,296425,0.948837,0.103448,0.051163,1.0,0.0,0.0,0.003813,0.042254,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6,0.083333,0.111111,0.001700,847770,0.976744,0.126437,0.023256,1.0,0.0,0.0,0.002267,0.136150,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.083333,0.047619,0.002179,646760,0.581395,0.080460,0.418605,1.0,0.0,0.0,0.002906,0.061033,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
18,0.083333,0.015873,0.001305,169471,0.693023,0.057471,0.306977,1.0,0.0,0.0,0.001740,0.023474,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
19,0.083333,0.047619,0.001364,233266,0.693023,0.080460,0.306977,1.0,0.0,0.0,0.001819,0.061033,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
X = df.drop(columns = ['tax_value'])
y = df.tax_value

In [8]:
skb = SelectKBest(f_regression, k = 5)

skb.fit(X, y)

SelectKBest(k=5, score_func=<function f_regression at 0x7ff2438c2dd0>)

In [9]:
skb_mask = skb.get_support()
X.columns[skb_mask]

Index(['bathrooms', 'area', 'total_rooms', 'property_size_category_medium',
       'property_size_category_large'],
      dtype='object')

In [10]:
lm = LinearRegression()

rfe = RFE(lm, n_features_to_select=3)

rfe.fit(X, y)

RFE(estimator=LinearRegression(), n_features_to_select=3)

In [11]:
rfe_mask = rfe.get_support()
X.columns[rfe_mask]

Index(['year_built', 'property_age', 'property_age_group_very_new'], dtype='object')